# PostgreSQLを操作する

In [1]:
from dotenv import load_dotenv
from psycopg_pool import ConnectionPool
import os

In [2]:
load_dotenv()

conninfo = os.environ["POSTGRES_CONNINFO"]

In [3]:
with ConnectionPool(conninfo=conninfo) as pool:
    with pool.connection() as conn:
        with conn.execute("select * from messages order by id asc") as cur:
            for record in cur:
                print(record)

(1, 'foo')
(2, 'bar')
(3, 'baz')
(4, 'qux')
